## Experiment tracking

(Example)

| ID | Data | Pre | Seperation Date | Algorithm | RMSLE Train | RMSLE Test | Code | Comments |
|---|---|---|---|---|---|---|---|---|
| 0 | gdsc2_public.csv | Standard | 01.01.2018 | Median | 1.62 | 2.10 | [Link](http://de-mucingode1/gitlab/dkuehlwein/global_data_science_challenge_2_public/tags/Baselines) | Baseline |
| 1 | gdsc2_public.csv | One-Hot-Encoding of 'work_type', 'work_priority' | 01.01.2018 | KNNRegressor (Default) | 1.99 | 1.49 | [Link](http://de-mucingode1/gitlab/dkuehlwein/global_data_science_challenge_2_public/tags/Baselines) | Performs worse than baseline on training. Model is not complex enough, need to check train/test split | 

In [4]:
%pip install --upgrade pandas-profiling

     |████████████████████████████████| 133kB 242kB/s eta 0:00:01
     |████████████████████████████████| 614kB 625kB/s eta 0:00:01
     |████████████████████████████████| 102kB 4.0MB/s ta 0:00:011
     |████████████████████████████████| 225kB 9.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 16.9MB/s eta 0:00:01


  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=148432 sha256=ca82569c0ac0111ab5f796e3e9eb4ea685fe7ac1010f2d02587bbbcf1c9f8090
  Stored in directory: /Users/zigfridzvezdin/Library/Caches/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27222 sha256=66b63465a57dcee3a4b58c546e98609f8c01c78f90a2dd9a39c5e1a8e089e905
  Stored in directory: /Users/zigfridzvezdin/Library/Caches/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for confuse: filename=confuse-1.0.0-cp36-none-any.whl size=19566 sha256=b331e46d0b65f715287ba53261ceedfe11f738994f9ab48b7e08321823ed4511
  Stored in directory: /Users/zigfridzvezdin/Library/Caches/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
Successfully built pandas-profiling htmlmin confuse
ERROR: jupyter-console 5.2.0 has requirement prompt_toolkit<2.0.0,>=1.0.0

In [1]:
import pandas as pd
import pandas_profiling as pp
import numpy as np

ImportError: cannot import name 'register_matplotlib_converters'

In [ ]:
df = pd.read_csv('Taxi-time Prediction Data/0. Airport data/Airport_Data.csv')

In [ ]:
df.head()

In [3]:
df.profile_report(style={'full_width':True})

AttributeError: 'DataFrame' object has no attribute 'profile_report'

In [5]:
pp.profile_report(df)

AttributeError: module 'pandas_profiling' has no attribute 'profile_report'

In [8]:
df = pd.DataFrame(
    np.random.rand(100, 5),
    columns=['a', 'b', 'c', 'd', 'e']
)
df.profile_report(style={'full_width':True})


AttributeError: 'DataFrame' object has no attribute 'profile_report'